# Objective

Interactively experiment with the graph dataloader, and test whether there is a good way to load graph attributes.

In [1]:
%cd GraphRNN

/home/alpaca/projects/Comp755-Project-GraphGeneration/GraphRNN


In [2]:
from main import *

In [3]:
args = Args()

In [4]:
G_list = Graph_load_batch(name='DD', node_attributes=False)

Loading graph dataset: DD
Loaded


In [5]:
G_list = [G for G in G_list if nx.number_connected_components(G) == 1]

In [6]:
dataset = Graph_sequence_sampler_pytorch_nodelabels(G_list, iteration=2000)

calculating max previous node, total iteration: 2000
iter 0 times
iter 400 times
iter 800 times
iter 1200 times
iter 1600 times
max previous node: 181


In [7]:
# need a way to consolidate into "unknown" or "UNK" labels
dataset.num_node_labels

21

In [8]:
dataset.n

903

In [9]:
dataset.node_lab_vocab

{11: 0,
 1: 1,
 8: 2,
 20: 3,
 7: 4,
 12: 5,
 4: 6,
 16: 7,
 10: 8,
 17: 9,
 2: 10,
 15: 11,
 3: 12,
 14: 13,
 6: 14,
 19: 15,
 9: 16,
 13: 17,
 5: 18,
 18: 19,
 65: 20,
 55: 20,
 78: 20,
 76: 20,
 34: 20,
 82: 20,
 58: 20,
 43: 20,
 64: 20,
 31: 20,
 59: 20,
 80: 20,
 56: 20,
 86: 20,
 41: 20,
 37: 20,
 27: 20,
 85: 20,
 88: 20,
 45: 20,
 35: 20,
 54: 20,
 71: 20,
 89: 20,
 25: 20,
 38: 20,
 33: 20,
 70: 20,
 29: 20,
 24: 20,
 39: 20,
 57: 20,
 77: 20,
 81: 20,
 30: 20,
 21: 20,
 69: 20,
 62: 20,
 50: 20,
 68: 20,
 84: 20,
 74: 20,
 66: 20,
 26: 20,
 44: 20,
 46: 20,
 47: 20,
 49: 20,
 75: 20,
 22: 20,
 28: 20,
 67: 20,
 79: 20,
 53: 20,
 36: 20,
 32: 20}

In [10]:
# check if all node labels are correctly one-hot-encoded
all([np.all(dataset[i]['labs'].sum(axis=1) == 1) for i in range(len(dataset))])

False

In [11]:
sample_strategy = torch.utils.data.sampler.WeightedRandomSampler([1.0 / len(dataset) for i in range(len(dataset))],
                                                                     num_samples=10, replacement=True)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=10, num_workers=0,
                                               sampler=sample_strategy, drop_last=True)

In [12]:
data = next(iter(data_loader))

In [13]:
data['x'].shape

torch.Size([10, 903, 181])

In [14]:
data['y'].shape

torch.Size([10, 903, 181])

In [15]:
data['labs'].shape

torch.Size([10, 903, 21])

In [16]:
data['len']

tensor([469, 218, 338, 180, 636, 435, 148, 305, 320, 336])

# Test training code

Recreate code in `train_rnn_epoch_nodelabs` in jupyter notebook, to aid in debugging.

In [ ]:
rnn = GRU_plain(
    input_size = dataset.max_prev_node, 
    embedding_size = args.embedding_size_rnn,
    hidden_size = args.hidden_size_rnn,
    num_layers = args.num_layers, 
    has_input = True, has_output = True,
    output_size = args.hidden_size_rnn_output
)         

output = GRU_plain(
    input_size = 1,
    embedding_size = args.embedding_size_rnn_output,
    hidden_size = args.hidden_size_rnn_output + args.node_embedding_size,
    num_layers = args.num_layers,
    has_input = True, has_output = True,
    output_size = 1
)

node_embed = MLP_plain(
    h_size = dataset.num_node_labels,
    embedding_size = args.node_embedding_size,
    y_size = args.node_embedding_size
)

node_pred = MLP_plain(
    h_size = args.hidden_size_rnn_output,
    embedding_size = args.node_embedding_size,
    y_size = dataset.num_node_labels
)

/home/alpaca/projects/Comp755-Project-GraphGeneration/GraphRNN/model.py:299: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(param,gain=nn.init.calculate_gain('sigmoid'))
/home/alpaca/projects/Comp755-Project-GraphGeneration/GraphRNN/model.py:297: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(param, 0.25)
/home/alpaca/projects/Comp755-Project-GraphGeneration/GraphRNN/model.py:302: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  m.weight.data = init.xavier_uniform(m.weight.data, gain=nn.init.calculate_gain('relu'))
/home/alpaca/projects/Comp755-Project-GraphGeneration/GraphRNN/model.py:337: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  m.weight.data = init.xavier_uniform(m.weight.data, gain=nn.init.calculate_gain('relu'))


Now run through a single iteration of the training (batch) for loop, testing to make sure the code works.

In [18]:
x_unsorted = data['x'].float()
y_unsorted = data['y'].float()
labs_unsorted = data['labs'].float()
y_len_unsorted = data['len']

In [19]:
y_len_max = max(y_len_unsorted)
x_unsorted = x_unsorted[:, 0:y_len_max, :]
y_unsorted = y_unsorted[:, 0:y_len_max, :]
labs_unsorted = labs_unsorted[:, 0:y_len_max, :]

In [20]:
# throws an error b/c no GPU, however can continue testing without this step.
# rnn.hidden = rnn.init_hidden(batch_size = x_unsorted.size(0))

In [21]:
y_len, sort_index = torch.sort(y_len_unsorted, 0, descending = True)
y_len = y_len.numpy().tolist()
x = torch.index_select(x_unsorted, 0, sort_index)
y = torch.index_select(y_unsorted, 0, sort_index)
labs = torch.index_select(labs_unsorted, 0, sort_index)

In [22]:
y_reshape = pack_padded_sequence(y, y_len, batch_first = True).data
labs_reshape = pack_padded_sequence(labs, y_len, batch_first = True).data
idx = [i for i in range(y_reshape.size(0)-1, -1, -1)]
idx = torch.LongTensor(idx)
y_reshape = y_reshape.index_select(0, idx)
labs_reshape = labs_reshape.index_select(0, idx)
y_reshape = y_reshape.view(*y_reshape.shape, 1)

In [23]:
output_x = torch.cat(
    (torch.ones(y_reshape.size(0), 1, 1), y_reshape[:, 0:-1, 0:1]),
    dim = 1
)
output_y = y_reshape
output_labs = labs_reshape

In [24]:
output_x.shape

torch.Size([3385, 181, 1])

In [25]:
output_y.shape

torch.Size([3385, 181, 1])

In [26]:
output_y_len = []
output_y_len_bin = np.bincount(np.array(y_len))
for i in range(len(output_y_len_bin)-1, 0, -1):
    count_temp = np.sum(output_y_len_bin[i:])
    output_y_len.extend([min(i, y.size(2))]*count_temp)

In [27]:
x = Variable(x)
y = Variable(y)
labs = Variable(labs)
output_x = Variable(output_x)
output_y = Variable(output_y)
output_labs = Variable(output_labs)

In [28]:
h = rnn(x, pack = True, input_len = y_len)
h = pack_padded_sequence(h, y_len, batch_first = True).data

In [29]:
idx = [i for i in range(h.size(0)-1, -1, -1)]
idx = Variable(torch.LongTensor(idx))
h = h.index_select(0, idx)

In [30]:
labs_pred = node_pred(h)
labs_pred = F.softmax(labs_pred, dim = 1)

In [31]:
labs_embedded = node_embed(output_labs)
hcat = torch.cat((h, labs_embedded), dim = 1)

In [32]:
hidden_null = Variable(torch.zeros(args.num_layers-1, *hcat.shape))
output.hidden = torch.cat((hcat.view(1, *hcat.shape), hidden_null), dim = 0)

In [33]:
y_pred = output(output_x, pack = True, input_len = output_y_len)
y_pred = torch.sigmoid(y_pred)

In [34]:
y_pred.shape

torch.Size([3385, 181, 1])

In [35]:
# so I don't actually think that this code is necessary...
# 
# y_pred = pack_padded_sequence(y_pred, output_y_len, batch_first = True)
# y_pred = pad_packed_sequence(y_pred, batch_first = True)[0]
# output_y = pack_padded_sequence(output_y, output_y_len, batch_first = True)
# output_y = pad_packed_sequence(output_y, batch_first = True)[0]

In [36]:
output_y.shape

torch.Size([3385, 181, 1])

In [37]:
loss_y = binary_cross_entropy_weight(y_pred, output_y)
loss_y

tensor(0.7449, grad_fn=<BinaryCrossEntropyBackward0>)

In [38]:
loss_labs = F.cross_entropy(labs_pred, output_labs)
loss_labs

tensor(3.0326, grad_fn=<DivBackward1>)

In [ ]:
# double check that this result is the same as the last one
loss_labs2 = F.cross_entropy(labs_pred, output_labs.argmax(dim = 1))
loss_labs2

In [39]:
loss = loss_y + loss_labs
loss.backward()

In [40]:
loss

tensor(3.7775, grad_fn=<AddBackward0>)

# Graph prediction

Test code in `test_rnn_epoch_nodelabs`, for graph prediction.

In [43]:
rnn = GRU_plain(
    input_size = dataset.max_prev_node, 
    embedding_size = args.embedding_size_rnn,
    hidden_size = args.hidden_size_rnn,
    num_layers = args.num_layers, 
    has_input = True, has_output = True,
    output_size = args.hidden_size_rnn_output
)         

output = GRU_plain(
    input_size = 1,
    embedding_size = args.embedding_size_rnn_output,
    hidden_size = args.hidden_size_rnn_output + args.node_embedding_size,
    num_layers = args.num_layers,
    has_input = True, has_output = True,
    output_size = 1
)

node_embed = MLP_plain(
    h_size = dataset.num_node_labels,
    embedding_size = args.node_embedding_size,
    y_size = args.node_embedding_size
)

node_pred = MLP_plain(
    h_size = args.hidden_size_rnn_output,
    embedding_size = args.node_embedding_size,
    y_size = dataset.num_node_labels
)

In [45]:
test_batch_size = 16
rnn.hidden = Variable(torch.zeros(args.num_layers, test_batch_size, args.hidden_size_rnn))

In [49]:
max_num_node = 900
y_pred_long = Variable(torch.zeros(test_batch_size, max_num_node, dataset.max_prev_node))
x_step = Variable(torch.ones(test_batch_size, 1, dataset.max_prev_node))

In [51]:
y_pred_long.shape

torch.Size([16, 900, 181])

In [52]:
x_step.shape

torch.Size([16, 1, 181])

In [48]:
print(args.max_num_node)

None


# Experimenting with BFS

In [48]:
adj = np.random.choice([0, 1], size=(10,10), p = [0.3, 0.7])
# adj = np.reshape(np.arange(100), (10,10))
adj

array([[0, 1, 0, 1, 1, 1, 1, 1, 1, 0],
       [1, 0, 0, 1, 1, 1, 1, 1, 1, 0],
       [1, 0, 1, 1, 0, 1, 1, 1, 1, 0],
       [0, 1, 0, 1, 0, 1, 1, 1, 1, 1],
       [1, 1, 1, 0, 0, 1, 1, 1, 1, 1],
       [1, 0, 1, 0, 1, 1, 0, 1, 1, 0],
       [1, 0, 1, 1, 1, 1, 0, 0, 1, 1],
       [1, 1, 1, 1, 0, 1, 0, 1, 0, 0],
       [1, 1, 1, 1, 0, 1, 0, 1, 1, 1],
       [1, 0, 1, 1, 1, 0, 0, 1, 1, 1]])

In [49]:
adj = np.tril(adj, k=-1)
adj

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 0, 0, 0, 0, 0],
       [1, 0, 1, 1, 1, 1, 0, 0, 0, 0],
       [1, 1, 1, 1, 0, 1, 0, 0, 0, 0],
       [1, 1, 1, 1, 0, 1, 0, 1, 0, 0],
       [1, 0, 1, 1, 1, 0, 0, 1, 1, 0]])

In [50]:
n = adj.shape[0]
adj = adj[1:n, 0:n-1]
adj

array([[1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 0, 0, 0, 0],
       [1, 0, 1, 1, 1, 1, 0, 0, 0],
       [1, 1, 1, 1, 0, 1, 0, 0, 0],
       [1, 1, 1, 1, 0, 1, 0, 1, 0],
       [1, 0, 1, 1, 1, 0, 0, 1, 1]])

In [51]:
# use max_prev_node to truncate
# note: now adj is a (n-1)*(n-1) matrix
max_prev_node = 10
adj_output = np.zeros((adj.shape[0], max_prev_node))
for i in range(adj.shape[0]):
    print(f'row {i}')
    input_start = max(0, i - max_prev_node + 1)
    input_end = i + 1
    print(f'input_start: {input_start}\tinput_end: {input_end}')
    output_start = max_prev_node + input_start - input_end
    output_end = max_prev_node
    print(f'output_start: {output_start}\toutput_end: {output_end}')
    adj_output[i, output_start:output_end] = adj[i, input_start:input_end]
    adj_output[i,:] = adj_output[i,:][::-1] # reverse order

row 0
input_start: 0	input_end: 1
output_start: 9	output_end: 10
row 1
input_start: 0	input_end: 2
output_start: 8	output_end: 10
row 2
input_start: 0	input_end: 3
output_start: 7	output_end: 10
row 3
input_start: 0	input_end: 4
output_start: 6	output_end: 10
row 4
input_start: 0	input_end: 5
output_start: 5	output_end: 10
row 5
input_start: 0	input_end: 6
output_start: 4	output_end: 10
row 6
input_start: 0	input_end: 7
output_start: 3	output_end: 10
row 7
input_start: 0	input_end: 8
output_start: 2	output_end: 10
row 8
input_start: 0	input_end: 9
output_start: 1	output_end: 10


In [52]:
adj_output

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 1., 0., 1., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 1., 1., 1., 1., 0., 0., 0.],
       [1., 0., 1., 0., 1., 1., 1., 1., 0., 0.],
       [1., 1., 0., 0., 1., 1., 1., 0., 1., 0.]])

In [53]:
adj_output = []
input_start = 0
for i in range(adj.shape[0]):
    print(f'row {i}')
    input_end = i + 1
    adj_slice = adj[i, input_start:input_end]
    adj_output.append(adj_slice)
    non_zero = np.nonzero(adj_slice)[0]
    input_start = input_end-len(adj_slice)+np.amin(non_zero)
    print(f'input_start: {input_start}\tinput_end: {input_end}')

row 0
input_start: 0	input_end: 1
row 1
input_start: 0	input_end: 2
row 2
input_start: 1	input_end: 3
row 3
input_start: 1	input_end: 4
row 4
input_start: 2	input_end: 5
row 5
input_start: 2	input_end: 6
row 6
input_start: 2	input_end: 7
row 7
input_start: 2	input_end: 8
row 8
input_start: 2	input_end: 9


In [54]:
adj_output

[array([1]),
 array([1, 0]),
 array([0, 1, 0]),
 array([1, 1, 0]),
 array([0, 1, 0, 1]),
 array([1, 1, 1, 1]),
 array([1, 1, 0, 1, 0]),
 array([1, 1, 0, 1, 0, 1]),
 array([1, 1, 1, 0, 0, 1, 1])]